# Pre-requisite
- We need to create a data lake account and also upload the input file to it
- Also need to create Service Principal and then allow access to SP on the Data Lake container which contains input files

## Mount Data from Data Lake

In [0]:
# Specify the Data Lake account details
adlsAccountName = "sadlmay22"
adlsContainerName = "inputdata"
adlsFolderName = "data"

In [0]:
# Specify details of SP which will be used in this notebook to connect to the Data Lake account
applicationId = "REDACTED"
authenticationKey = "REDACTED"
tenandId = "REDACTED"

In [0]:
endpoint = "https://login.microsoftonline.com/" + tenandId + "/oauth2/token"
source = "abfss://" + adlsContainerName + "@" + adlsAccountName + ".dfs.core.windows.net/"

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": applicationId,
           "fs.azure.account.oauth2.client.secret": authenticationKey,
           "fs.azure.account.oauth2.client.endpoint": endpoint}

In [0]:
dbutils.fs.unmount("/mnt/data")
print("Source: ", source)
print("configs: ", configs)
print("endpoint: ", endpoint)

print("applicationId: ", applicationId)
print("authenticationKey: ", authenticationKey)
print("tenandId: ", tenandId)

print("adlsAccountName: ", adlsAccountName)
print("adlsContainerName: ", adlsContainerName)

print("adlsFolderName: ", adlsFolderName)

In [0]:
try:
  dbutils.fs.mount(source = source,mount_point = "/mnt/data",extra_configs = configs)
except Exception as e:
  print(e)
  #pass

In [0]:
%fs
ls /mnt/data/

In [0]:
from pyspark.sql.functions import col, column, spark_partition_id

In [0]:
flightData2015="dbfs:/mnt/data/data/flight-data/csv/2010-summary.csv"
flightData2015="dbfs:/mnt/data/data/flight-data/csv"

In [0]:
flightDataDF2015 = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv(flightData2015)

flightDataDF2015=flightDataDF2015.withColumn("partitionID", spark_partition_id())
flightDataDF2015.rdd.getNumPartitions()

In [0]:
flightDataDF2015.show(5000)

In [0]:
from pyspark.sql.functions import concat, collect_list, collect_set, concat_ws

#Notice that values in the column - DEST_COUNTRY_NAME are spead over all partitions. It will create shuffling issues if we do any operation based on DEST_COUNTRY_NAME
flightDataDF2015.groupBy("DEST_COUNTRY_NAME")\
.agg(concat_ws(", ", collect_set(flightDataDF2015.partitionID))).display(100)

## Repartition based on a column using which we need to do group by

In [0]:
partition_count=flightDataDF2015.select('DEST_COUNTRY_NAME').distinct().count()
print(partition_count)

In [0]:
flightDataDF2015_rp=flightDataDF2015.repartition(partition_count, col("DEST_COUNTRY_NAME")).withColumn("partitionID", spark_partition_id())
flightDataDF2015_rp.sort("DEST_COUNTRY_NAME").show(5000)

In [0]:
flightDataDF2015_rp.sort("partitionID").show(5000)

In [0]:
flightDataDF2015_rp.rdd.getNumPartitions()

In [0]:
# Notice that one column is aligned to a single partition. It will avoid shuffling issues
flightDataDF2015_rp.groupBy("DEST_COUNTRY_NAME")\
.agg(concat_ws(", ", collect_set(flightDataDF2015_rp.partitionID))).display(5000)

#flightDataDF2015_rp.sort("partitionID").show(20000)

In [0]:
# Notice that one column is aligned to a single partition. It will avoid shuffling issues
flightDataDF2015_rp.groupBy("partitionID")\
.agg(concat_ws(", ", collect_set(flightDataDF2015_rp.partitionID))).display(5000)

#flightDataDF2015_rp.sort("partitionID").show(20000)

## Execution plan comparision between SQL Way and Datafram Way

In [0]:
flightDataDF2015.createOrReplaceTempView("flight_data_2015")
sqlWay = spark\
.sql("""
select dest_country_name, count(1)
from flight_data_2015
group by dest_country_name
""")

In [0]:
dataFrameWay = flightDataDF2015\
.groupBy("dest_country_name")\
.count()

In [0]:
# The physical execution plans are almost ame either way
sqlWay.explain()
dataFrameWay.explain()

## Grouping on regular dataframe

In [0]:
flightDataDF2015_groupBy=flightDataDF2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")

In [0]:
# Only 3 partitions are created and group by will result in shuffling
flightDataDF2015.rdd.getNumPartitions()

In [0]:
# Let's save the results. This will initiate Action and also to the data shuffling
dbutils.fs.rm("/mnt/data/data/gprby_DEST_COUNTRY_NAME",True)
#Here data shuffleing will happen which is an expensive operation
flightDataDF2015_groupBy.write.mode('overwrite').format("csv").option("header", "true").save("dbfs:/mnt/data/data/gprby_DEST_COUNTRY_NAME")

In [0]:
%fs
ls /mnt/data/data/gprby_DEST_COUNTRY_NAME

## Grouping on repartitioned dataframe

In [0]:
# No/Minimal shuffling will happen
flightDataDF2015_rp_groupBy=flightDataDF2015_rp\
.groupBy("dest_country_name")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")

In [0]:
dbutils.fs.rm("/mnt/data/data/gprby2_DEST_COUNTRY_NAME",True)

In [0]:
# Here the RDD Action will trigger and notice that multiple files will be saved, one file each partition
flightDataDF2015_rp_groupBy.write.mode('overwrite').format("csv").option("header", "true").save("dbfs:/mnt/data/data/gprby2_DEST_COUNTRY_NAME")

In [0]:
%fs
ls /mnt/data/data/gprby2_DEST_COUNTRY_NAME

## Dealing with Manual / Static Schema

In [0]:
staticSchema = flightDataDF2015.schema

In [0]:
print(staticSchema)

In [0]:
flightDataDF2015 = spark\
.read\
.schema(staticSchema)\
.option("header", "true")\
.csv(flightData2015)

In [0]:
flightDataDF2015.display()

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

In [0]:
myManualSchema = StructType([\
                            StructField("DEST_COUNTRY_NAME", StringType(), nullable=False),\
                            StructField("ORIGIN_COUNTRY_NAME", StringType(), nullable=False),\
                            StructField("count", StringType(), nullable=False, metadata={"hello":"world"})\
                            ])
df1 = spark.read.format("csv").schema(myManualSchema)\
.load(flightData2015)


In [0]:
df1.display()

In [0]:
flightDataDF2015.rdd.getNumPartitions()

## Can specify to create multiple files, one for each value in the columns

In [0]:
# Slow Performance as all data is to be reshuffled
dbutils.fs.rm("dbfs:/mnt/data/data/P_By_DEST_COUNTRY_NAME", True)
flightDataDF2015.write.partitionBy("DEST_COUNTRY_NAME").format("csv").save("dbfs:/mnt/data/data/P_By_DEST_COUNTRY_NAME")

In [0]:
%fs
ls /mnt/data/data/P_By_DEST_COUNTRY_NAME

In [0]:
# Fast Performance as all data is spread across partitions with each executer
dbutils.fs.rm("dbfs:/mnt/data/data/P_By_DEST_COUNTRY_NAME_2", True)
flightDataDF2015_rp.write.partitionBy("DEST_COUNTRY_NAME").format("csv").save("dbfs:/mnt/data/data/P_By_DEST_COUNTRY_NAME_2")

In [0]:
%fs
ls /mnt/data/data/P_By_DEST_COUNTRY_NAME_2

In [0]:
# Multiple executers will load the data paralally
df1 = spark.read.format("csv").schema(myManualSchema)\
.load("dbfs:/mnt/data/data/P_By_DEST_COUNTRY_NAME_2")

In [0]:
# Multiple executers will load the data paralally
df1 = spark.read.format("csv").schema(myManualSchema)\
.load("dbfs:/mnt/data/data/P_By_DEST_COUNTRY_NAME_2")

In [0]:
df1.createOrReplaceTempView("flight_data_2015")
sqlWay = spark\
.sql("""
select *
from flight_data_2015
where dest_country_name='United States'
""")

In [0]:
sqlWay.show(10000000)

In [0]:
# Notice the number of partitions created
df1.rdd.getNumPartitions()

In [0]:
df1.display()